# POC Power BI TS Formation

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import xlsxwriter

## Import des données

In [22]:
%%time
tcw = pd.read_csv(r'Data/TrainingCollectiveWishes.csv', engine=None,encoding='utf-8', sep=';') # Demandes collectives non affectées à un plan de formation
tiw = pd.read_csv(r'Data/TrainingIndividualWishes.csv', engine=None,encoding='utf-8', sep=';') # Demandes individuelles non affectées à un plan de formation
tp = pd.read_csv(r'Data/TrainingPlan.csv', engine=None,encoding='utf-8', sep=';') # Plans de formation
tpcw = pd.read_csv(r'Data/TrainingPlanCollectiveWishes.csv', engine=None,encoding='utf-8', sep=';') # Demande de formation collectives prises en charge dans un plan de formation
tpci = pd.read_csv(r'Data/TrainingPlanIndividualWishes.csv', engine=None,encoding='utf-8', sep=';') # Demande de formation individuelles prises en charge dans un plan de formation
tr = pd.read_csv(r'Data/TrainingRegister.csv', engine=None,encoding='utf-8', sep=';') # Table recensant les inscription aux sessiosn de formation 
ts = pd.read_csv(r'Data/TrainingSessions.csv', engine=None,encoding='utf-8', sep=';') # Table recensant les sessions de formation 
tsc = pd.read_csv(r'Data/TrainingStageCost.csv', engine=None,encoding='utf-8', sep=';') # Table contenant les coûts des stages
tsv2 = pd.read_csv(r'Data/TrainingStagev2.csv', engine=None,encoding='utf-8', sep=';') # Table contenant les stages
emp = pd.read_csv(r'Data/Employees.csv', engine=None,encoding='utf-8', sep=';') # Table maître employé
empc = pd.read_csv(r'Data/EmployementContract.csv', engine=None,encoding='utf-8', sep=';') # Table contrat employé
indO = pd.read_csv(r'Data/IndividualOrganization.csv', engine=None,encoding='utf-8', sep=';') # Table organisation employé
indpp = pd.read_csv(r'Data/IndivPPCsNew.csv', engine=None,encoding='utf-8', sep=';') # ?
para = pd.read_excel(r'Data/Paramétrage.xlsx', sheets="Mapp_data") # Table de paramétrage servant à mapper les données

Wall time: 187 ms


## Fonctions exploitées dans le code

### Fonctions pour mapper les données

In [23]:
def dict_mapp(colonne_name):
    df = para  
    df = para.loc[para.Colonne == colonne_name]
    dic = {}
    df.reset_index(drop=True, inplace=True)
    print("1" + str(colonne_name))
    print(df)
    for i, val in enumerate(df.iterrows()):
        dic[df.at[i, "Valeur source"]] = df.at[i, "Valeur cible"]
        print("2" + str(colonne_name))
    return dic

def df_mapp(dataframe):
    col_map = para["Colonne"].unique()    
    for col in col_map:
        if col in dataframe.columns:
            dataframe[col] = dataframe[col].map(dict_mapp(col))
    return dataframe   

## Renommage des clés

In [24]:
ts.rename(columns={'clientcode':'SessionID', 'coursecode':'FormationID', 'startdate':'SessionDate'},inplace=True)
tsv2.rename(columns={'clientcode':'SessionID'},inplace=True)
tr.rename(columns={'clientcode':'SessionID', 'traineusername':'USERNAME'},inplace=True)
tp.rename(columns={'plancode':'PlanID'},inplace=True)
emp.rename(columns={'username':'USERNAME'}, inplace=True)
tcw.rename(columns={'clientcode':'WishID','coursecode':'FormationID','username':'USERNAME'},inplace=True)
tiw.rename(columns={'username':'USERNAME', 'employeenumber':'ZY00.MATCLE', 'clientcode':'WishID', 'coursecode':'FormationID'},inplace=True)
tpcw.rename(columns={'plan_code':'PlanID', 'wish_code':'WishID'}, inplace=True)
tpci.rename(columns={'plan_code':'PlanID', 'wish_code':'WishID'}, inplace=True)

In [25]:
my_dict_columns = {"tcw": tcw.columns, "tiw": tiw.columns, "tp" : tp.columns, "tpcw" : tpcw.columns, "tr" : tr.columns, "ts" : ts.columns, "tsc" : tsc.columns, "tsv2" : tsv2.columns, "emp" : emp.columns,
           "empc" : empc.columns, "indO": indO.columns, "indpp": indpp.columns}

my_dict_dataframe = {"tcw": tcw, "tiw": tiw, "tp" : tp, "tpcw" : tpcw, "tr" : tr, "ts" : ts, "tsc" : tsc, "tsv2" : tsv2, "emp" : emp,
           "empc" : empc, "indO": indO, "indpp": indpp}

In [26]:
def get_key(val): 
    res=[]
    for key, value in my_dict_columns.items():        
        if val in(value): 
            res.append(key)     
    return res
#get_key("plan_code")

## Creation des tables de dimension

### Création de la table dimension demandes de formations

In [27]:

wish_table = tcw.append(tiw,sort=True)
wish_table.reset_index(drop=True, inplace=True)
wish_table["Wish_key"] = wish_table.index
wish_table.loc[wish_table.nbtrainees.isna(), "nbtrainees"] = 1
wish_table = df_mapp(wish_table)
my_dict_dataframe["wish_table"]= wish_table
#wish_table.to_excel(r'Transformed data\wish_table.xlsx')

1state
   Colonne Valeur source Valeur cible
0    state      Souhaite     Souhaité
1    state      Souhaité     Souhaité
2    state       Accepte      Accepté
3    state       Accepté      Accepté
4    state        Refuse       Refusé
5    state        Refusé       Refusé
6    state        Valide       Validé
7    state        Validé       Validé
8    state       Propose      Proposé
9    state       Proposé      Proposé
10   state        Annule       Annulé
11   state        Annulé       Annulé
2state
2state
2state
2state
2state
2state
2state
2state
2state
2state
2state
2state
1creationmode
        Colonne Valeur source Valeur cible
0  creationmode        Wished     Souhaité
1  creationmode      Proposed      Proposé
2creationmode
2creationmode


## Création des tables de fait

### Table de fait demande de formation

In [28]:
wish_fact = wish_table[["Wish_key", "nbtrainees"]].copy()
wish_fact.loc[:, "Nb_demandes"]= 1

my_dict_dataframe["wish_fact"] = wish_fact

## Export des données vers le répertoire Transformed data

In [29]:
for key, value in my_dict_dataframe.items():
        location = "Transformed data"
        file_name = str(key) + '.csv'
        location = os.path.join(location, file_name)  
        value.to_csv(location, encoding='utf16', index=False)